# Library Imports
Importing the necessary libraries. 

In [1]:
import pandas as pd
import zipfile 
import pickle
import os

# Data Analysis
We are separately reading/extracting the `data.zip`, `invoices_new.pkl` and `expired_invoices.txt` files, to understand
how they look like, what features and data types do they present. This steps helps us achieve a better understanding about the upcoming issues that will be resolved in the class creation process (such as dealing with numbers as strings which have to be converted to integers , e.g. invoices_id)

In [2]:
data_path = 'data.zip'
directory_to_extract  = os.path.join(os.getcwd(), 'data')
with zipfile.ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract)

In [3]:
invoices_new_path = os.path.join(directory_to_extract, 'invoices_new.pkl')
df = pd.read_pickle(invoices_new_path)
print(f'The new invoices dataframe has a data type of {type(df)}')
df

The new invoices dataframe has a data type of <class 'list'>


[{'id': '365371O',
  'created_on': '2019-05-27',
  'items': [{'item': {'id': 142446,
     'name': 'ii_142446',
     'unit_price': 182,
     'type': 1},
    'quantity': 6},
   {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1},
    'quantity': 1}]},
 {'id': 389528,
  'created_on': '2019-03-19',
  'items': [{'item': {'id': 172535,
     'name': 'ii_172535',
     'unit_price': 105,
     'type': 0},
    'quantity': 4},
   {'item': {'id': 175208, 'name': 'ii_175208', 'unit_price': 109, 'type': 0},
    'quantity': 7},
   {'item': {'id': 139970, 'name': 'ii_139970', 'unit_price': 193, 'type': 2},
    'quantity': 4},
   {'item': {'id': 156273, 'name': 'ii_156273', 'unit_price': 129, 'type': 2},
    'quantity': 6},
   {'item': {'id': 178512, 'name': 'ii_178512', 'unit_price': 108, 'type': 1},
    'quantity': 7},
   {'item': {'id': 191045, 'name': 'ii_191045', 'unit_price': 195, 'type': 1},
    'quantity': 8},
   {'item': {'id': 189841, 'name': 'ii_189841', 'unit_price': 1

In [4]:
expired_invoices_path = os.path.join(directory_to_extract, 'expired_invoices.txt')
ls = []
with open(expired_invoices_path, 'r') as file:
    read_lines = file.readlines()
    for i in read_lines:
        ls.extend([int(x.strip()) for x in i.split(',')])

expired_invoices = ls
print(f'The expired invoices have a data type of {type(expired_invoices)}')
print('Expired Invoices:', expired_invoices)

The expired invoices have a data type of <class 'list'>
Expired Invoices: [305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299]


# Class Creation
- invoice_id: int
- created_on: datetime64[ns]
- invoiceitem_id: int
- invoiceitem_name: str
- type: str (use this conversion table: {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'})
- unit_price: int
- total_price: int (unit_price*quantity)
- percentage_in_invoice: float (unit_price*quantity / invoice_total)
- is_expired: bool (whether invoice_id is contained expired_invoices.txt or not)


In [5]:
class DataExtractor:
    def __init__(self, data_path):
        # Initializing the directories based on the inputted data path
        self.data_path = data_path
        self.expired_invoices_path = os.path.join(os.getcwd(), 'data', 'expired_invoices.txt')

    def load(self):
        # Extracting the initial ZIP file 
        directory_to_extract = os.path.join(os.getcwd(), 'data')
        with zipfile.ZipFile(self.data_path, 'r') as zip_ref:
            zip_ref.extractall(directory_to_extract)

        # Extracting new invoices
        invoices_new_path = os.path.join(directory_to_extract, 'invoices_new.pkl')
        invoices_list = pd.read_pickle(invoices_new_path)

        data = []
        # Since the initial form of the invoices file is a dictionary inside the list, we iterate over it (we have seen this in Data Analysis part)
        for dict in invoices_list: 
            invoice_id = dict.get('id', None)
            created_on = dict.get('created_on', None)

            # Avoiding the date time error for "created_on" feature
            try:
                created_on = pd.to_datetime(created_on)
            except ValueError:
                continue

            
            items = dict.get('items', [])
            for item in items:
                item_info = item.get('item', {})
                data.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': item_info.get('id', None),
                    'invoiceitem_name': item_info.get('name', None),
                    'type': item_info.get('type', 'Unknown'),
                    'unit_price': item_info.get('unit_price', None),
                    'total_price': item_info.get('unit_price', None) * item.get('quantity', None)
                })

        self.df = pd.DataFrame(data)

        # Extracting expired invoices
        expired_invoices = []
        with open(self.expired_invoices_path, 'r') as file:
            read_lines = file.readlines()
            for line in read_lines:
                expired_invoices.extend([int(x.strip()) for x in line.split(',')])
        self.expired_invoices = expired_invoices

    def transform(self):
        dff = self.df.copy()
        # Creating "total_price" which depends on the initial features loaded beforehand
        dff['total_price'] = pd.to_numeric(dff['total_price'], errors='coerce')
    
        # Calculating the total invoice price (sum of all total prices)
        total_invoice_price = dff['total_price'].sum()

        # Calculating the percentage in invoice based on total price/invoice total price
        dff['percentage_in_invoice'] = dff['total_price'] / total_invoice_price

        # Looking for expired invoices
        dff['is_expired'] = dff['invoice_id'].isin(self.expired_invoices)

        # Mapping the provided conversions of the types
        types = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}
        dff['type'] = dff['type'].map(types)

        # Filter out rows with invalid invoice_id values
        # Handling base 10 string with str.isdigit() method
        dff = dff[dff['invoice_id'].astype(str).str.isdigit()]

        # Handling NaN values
        dff['total_price'] = dff['total_price'].fillna(0)
        
        # Handling data types
        dff['invoice_id'] = dff['invoice_id'].astype(int)
        dff['created_on'] = pd.to_datetime(dff['created_on'])
        dff['invoiceitem_id'] = dff['invoiceitem_id'].astype(int)
        dff['invoiceitem_name'] = dff['invoiceitem_name'].astype(str)
        dff['unit_price'] = dff['unit_price'].astype(int)
        dff['total_price'] = dff['total_price'].astype(int)
        dff['percentage_in_invoice'] = dff['percentage_in_invoice'].astype(float)
        dff['is_expired'] = dff['is_expired'].astype(bool)

        # Sorting accordingly
        dff = dff.sort_values(by=['invoice_id', 'invoiceitem_id'], ascending=[True, True])

        return dff


# Class Usage

In [6]:
data_path = 'data.zip'
data_extractor = DataExtractor(data_path)

In [7]:
data_extractor.load()
results = data_extractor.transform()

In [8]:
results.head()

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice,is_expired
292,301695,2019-04-26,103215,ii_103215,Other,135,945,0.002897,False
290,301695,2019-04-26,166227,ii_166227,Equipment,118,708,0.002170,False
293,301695,2019-04-26,171394,ii_171394,Other,128,128,0.000392,False
291,301695,2019-04-26,195625,ii_195625,Other,148,740,0.002268,False
130,304245,2019-03-17,121446,ii_121446,Service,158,632,0.001937,False


Below we see that all required features are created with their corresponding data types. It is time to save the results./

In [9]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 430 entries, 292 to 396
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   invoice_id             430 non-null    int64         
 1   created_on             430 non-null    datetime64[ns]
 2   invoiceitem_id         430 non-null    int64         
 3   invoiceitem_name       430 non-null    object        
 4   type                   427 non-null    object        
 5   unit_price             430 non-null    int64         
 6   total_price            430 non-null    int64         
 7   percentage_in_invoice  426 non-null    float64       
 8   is_expired             430 non-null    bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 30.7+ KB


# Saving the results as a .csv file

In [10]:
results.to_csv("results.csv", index = False)

# References
- https://stackoverflow.com/questions/3451111/unzipping-files-in-python
- https://stackoverflow.com/questions/24906126/how-to-unpack-pkl-file
- https://stackoverflow.com/questions/40096612/how-do-i-open-a-text-file-in-python
- https://www.tutorialspoint.com/python/string_isdigit.htm